In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


from astropy.io import fits




In [2]:
# Assembly task
# https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/isrTask.py

from lsst.ip.isr.assembleCcdTask import (AssembleCcdConfig, AssembleCcdTask)
from lsst.ip.isr.isrTask import (IsrTask, IsrTaskConfig)

#https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/overscan.py
from lsst.ip.isr import  OverscanCorrectionTaskConfig, OverscanCorrectionTask

In [3]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
! eups list -s

afw                   g29afe694f1+a272651890 	current w_2023_40 setup
alert_packet          g6041bd2d66 	current w_2023_40 setup
analysis_drp          g8a53795882+0c0064919b 	current w_2023_40 setup
analysis_tools        g01ff57b274+b4ae54b57f 	current w_2023_40 setup
ap_association        g9b2d7c2ad1+0898bae26d 	current w_2023_40 setup
ap_pipe               g9f065385ad+72c0e07383 	current w_2023_40 setup
ap_verify             g60239a501b+d279a3926c 	current w_2023_40 setup
astro_metadata_translator g0978aabd1f+0ffa813159 	current w_2023_40 setup
astshim               g3273194fdb+f6908454ef 	current w_2023_40 setup
atmospec              LOCAL:/home/m/monroy/repos/repos_w_2023_35/atmospec 	setup
base                  g98ffbb4407+81bc2a20b4 	current w_2023_40 setup
cbp                   ga32fcf49c8+650be1a39c 	current w_2023_40 setup
cell_coadds           g9faf7ed3f3+0c0064919b 	current w_2023_40 setup
coadd_utils           g337abbeb29+650be1a39c 	current w_2023_40 setup
cp_pipe         

# Butler 

In [6]:
import lsst.daf.butler as dafButler

In [7]:
embargo = True

In [8]:
if embargo:
    repo="/sdf/group/rubin/repo/oga/"
else:
    repo = "/sdf/group/rubin/repo/main"
butler = dafButler.Butler(repo)
registry = butler.registry

In [9]:
cameraName = 'LATISS'
calibCollections = ['LATISS/defaults','LATISS/raw/all']
for col in registry.queryCollections("*LATISS/calib*"):
    #print(col)
    calibCollections.append(col)

We check the different dimension records existing for *physical_filter*: 

In [10]:
phys_filters = list(registry.queryDimensionRecords('physical_filter',where="instrument='LATISS'"))

Print physical_filters with hologram in place: 

In [11]:
for f_ in phys_filters:
    if 'holo' in f_.name:
        print(f_.name)

empty~holo4_003
blank_bk7_wg05~holo4_003
KPNO_1111_436nm~holo4_003
KPNO_373A_677nm~holo4_003
KPNO_406_828nm~holo4_003
unknown~holo4_003
BG40~holo4_003
quadnotch1~holo4_003
RG610~holo4_003
FELH0600~holo4_003
SDSSg~holo4_003
SDSSr~holo4_003
SDSSi~holo4_003
BG40_65mm_1~holo4_003
BG40_65mm_2~holo4_003
OG550_65mm_1~holo4_003
OG550_65mm_2~holo4_003
SDSSg_65mm~holo4_003
SDSSr_65mm~holo4_003
SDSSi_65mm~holo4_003
SDSSu_65mm~holo4_003
SDSSz_65mm~holo4_003
SDSSy_65mm~holo4_003
empty~holo4_001
blank_bk7_wg05~holo4_001
KPNO_1111_436nm~holo4_001
KPNO_373A_677nm~holo4_001
KPNO_406_828nm~holo4_001
unknown~holo4_001
BG40~holo4_001
BG40_65mm_1~holo4_001
BG40_65mm_2~holo4_001
quadnotch1~holo4_001
RG610~holo4_001
OG550_65mm_1~holo4_001
OG550_65mm_2~holo4_001
FELH0600~holo4_001
SDSSg~holo4_001
SDSSr~holo4_001
SDSSi~holo4_001
SDSSu_65mm~holo4_001
SDSSg_65mm~holo4_001
SDSSr_65mm~holo4_001
SDSSi_65mm~holo4_001
SDSSz_65mm~holo4_001
SDSSy_65mm~holo4_001
collimator~holo4_003
collimator~holo4_001
cyl_lens~holo4_0

# Selection of exposures with cylindrical lens 

In [12]:
obs_type = 'science'
physical_filter = 'cyl_lens~holo4_003' #Note that there are no exposure with this config for the moment 

In [13]:
df_science = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start',
                                    'time_end' ,'type', 'target','filter_disp','zenith_angle',
                                    'exp_time','ra','dec','skyangle','science_program'])

In [14]:

where_exps = "instrument='LATISS' AND exposure.observation_type='{0}' AND physical_filter='{1}'".format(obs_type,physical_filter)
#where_exps = "instrument='LATISS' AND exposure.observation_type='{0}' AND exposure.day_obs={1}".format(obs_type,obs_day)
for i, info in enumerate(registry.queryDimensionRecords('exposure',where=where_exps)):
    
    #if info.observation_type=='science':
    id_ = info.id
    obs_id_ = info.obs_id
    day_obs_ = info.day_obs
    seq_num_ = info.seq_num
    timespan_ = info.timespan
    timespan_begin_ = pd.to_datetime(timespan_.begin.to_string())
    timespan_end_ = pd.to_datetime(timespan_.end.to_string())
    timespan_begin_jd_ = timespan_.begin.jd
    timespan_begin_mjd_ = timespan_.begin.mjd

    observation_type_ = info.observation_type
    target_name_ = info.target_name
    physical_filter_ = info.physical_filter
    zenith_angle_ = info.zenith_angle
    exposure_time_ = info.exposure_time
    tracking_ra_ = info.tracking_ra
    tracking_dec_ = info.tracking_dec
    sky_angle_ = info.sky_angle
    science_program_ = info.science_program

    df_science.loc[i] = [id_, obs_id_, day_obs_, seq_num_,timespan_begin_,timespan_end_ ,observation_type_, \
                         target_name_, physical_filter_, zenith_angle_, exposure_time_,tracking_ra_, \
                         tracking_dec_, sky_angle_, science_program_]

    if i < 2:
        print(i)
        print(info)
        print("\t timespan:            ",info.timespan)
        print("\t timespan.begin:      ",info.timespan.begin)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        print("-----------------------------------------------------")
    

0
exposure:
  instrument: 'LATISS'
  id: 2023091200125
  physical_filter: 'cyl_lens~holo4_003'
  obs_id: 'AT_O_20230912_000125'
  exposure_time: 90.0
  dark_time: 90.2414
  observation_type: 'science'
  observation_reason: 'cyl_lens'
  day_obs: 20230912
  seq_num: 125
  seq_start: 125
  seq_end: 125
  group_name: '2023-09-12T23:45:36.165'
  group_id: 2744919361650000
  target_name: 'gam Gru'
  science_program: 'SITCOM-1009'
  tracking_ra: 328.47710063386694
  tracking_dec: -37.366563075205136
  sky_angle: 353.88281242076397
  azimuth: 113.56899044390968
  zenith_angle: 42.05053957893198
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2023-09-12 23:50:10.329008', scale='tai', format='iso'), end=astropy.time.Time('2023-09-12 23:51:40.570000', scale='tai', format='iso'))
	 timespan:             [2023-09-12T23:50:10.329008, 2023-09-12T23:51:40.570000)
	 timespan.begin:       2023-09-12 23:50:10.329008
	 id:                   2023091200125
	 day_obs:              202309

In [15]:
print('Number of entries = ',len(df_science))

Number of entries =  32


In [16]:
df_science.tail(5)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter_disp,zenith_angle,exp_time,ra,dec,skyangle,science_program
27,2023092700208,AT_O_20230927_000208,20230927,208,2023-09-27 23:47:14.036019,2023-09-27 23:52:14.283,science,del Cap,cyl_lens~holo4_003,34.725847,300.0,326.757926,-16.127419,330.393018,SITCOM-1009
28,2023092700209,AT_O_20230927_000209,20230927,209,2023-09-27 23:52:37.992017,2023-09-27 23:57:38.222,science,del Cap,cyl_lens~holo4_003,33.609815,300.0,326.760403,-16.126080,330.393008,SITCOM-1009
29,2023092700210,AT_O_20230927_000210,20230927,210,2023-09-27 23:57:50.909008,2023-09-28 00:02:51.154,science,del Cap,cyl_lens~holo4_003,32.539513,300.0,326.760427,-16.126058,330.393083,SITCOM-1009
30,2023092700211,AT_O_20230927_000211,20230927,211,2023-09-28 00:03:14.059981,2023-09-28 00:08:14.311,science,del Cap,cyl_lens~holo4_003,31.433937,300.0,326.755358,-16.128835,330.393186,SITCOM-1009
31,2023092700212,AT_O_20230927_000212,20230927,212,2023-09-28 00:08:27.095017,2023-09-28 00:13:27.336,science,del Cap,cyl_lens~holo4_003,30.373533,300.0,326.755421,-16.128792,330.393123,SITCOM-1009


### Available dates: 

In [17]:
obs_days = np.unique(np.sort(np.array(df_science.day_obs)))
print(obs_days)

[20230912 20230913 20230914 20230927]


### Available exposures for selected date: 

In [18]:
obs_day = 20230914
df_selected = df_science[df_science.day_obs==obs_day]

In [19]:
seq_nums = np.sort(np.array(df_selected.seq_num))
print(seq_nums)

[31 32 33 34 35 36 37 38]


In [20]:
ids = np.sort(np.array(df_selected.id))
print(ids)

[2023091400031 2023091400032 2023091400033 2023091400034 2023091400035
 2023091400036 2023091400037 2023091400038]
